In [2]:
from pulp import *


In [3]:
I = ["NY", "Rm", "Mh", "Ba", "Va"]  # set of facilities
J = ["Missi", "Bramt"]  # set of DCs
W = ["NY_reg", "NY_half"] # set of options for NY
F = {
    "NY": {'NY_reg':315, 'NY_half':157.5},    
    "Rm": 305,
    "Mh": 295,
    "Ba": 160,
    "Va": 200,
}#Fixed cost of facilities
C = {
    "NY": {'NY_reg':12.5, 'NY_half':6.25}, 
    "Rm": 15,
    "Mh": 21,
    "Ba": 27.5,
    "Va": 25,
}
SC = {
    'NY': {'Missi': 5, 'Bramt': 6},
    'Rm': {'Missi': 7, 'Bramt': 8},
    'Mh': {'Missi': 12, 'Bramt': 9},
    'Ba': {'Missi': 17, 'Bramt': 15},
    'Va': {'Missi': 4, 'Bramt': 3}
}
D={'Missi':29.5, 'Bramt':12.5} # demands of the DCs

In [4]:
model = LpProblem("Production_Facilities_Multiple", LpMinimize)

In [5]:
x = LpVariable.dicts("x", (I, J), lowBound=0, cat='Continuous')
y = LpVariable.dicts("y", (I, W), 0, 1, LpBinary)

In [6]:
model += lpSum(F[i][w] * y[i][w] for i in I for w in W if i == "NY") + lpSum(F[i] * y[i][w] for i in I for w in W if i != "NY") + lpSum(SC[i][j] * x[i][j] for i in I for j in J), "Objective"

In [14]:
for i in I:
    if i == "NY":
        model += lpSum(y[i][w] for w in W) <= 1, f"one should be chosen_{i}"

In [18]:
for j in J:
    model += lpSum(x[i][j] for i in I) == 1, f"every DC {j} is served by at least one facility"

In [19]:
for i in I:
    model += lpSum(D[j] * x[i][j] for j in J) <= lpSum(F[i][w] * y[i][w] for i in I for w in W if i == "NY") + lpSum(F[i] * y[i][w] for i in I for w in W if i != "NY"), f"Satisfy_Capacity_and_Demand{i}"

In [20]:
for j in J:
    for i in I:
        model += x[i][j] <= lpSum(y[i][w] for w in W), f"m-Must be open to supply {i}{j}"

In [24]:
model.writeLP('Production_Facilities_Multiple_2nd_Trial')

[x_Ba_Bramt,
 x_Ba_Missi,
 x_Mh_Bramt,
 x_Mh_Missi,
 x_NY_Bramt,
 x_NY_Missi,
 x_Rm_Bramt,
 x_Rm_Missi,
 x_Va_Bramt,
 x_Va_Missi,
 y_Ba_NY_half,
 y_Ba_NY_reg,
 y_Mh_NY_half,
 y_Mh_NY_reg,
 y_NY_NY_half,
 y_NY_NY_reg,
 y_Rm_NY_half,
 y_Rm_NY_reg,
 y_Va_NY_half,
 y_Va_NY_reg]

In [22]:
model.solve()

1

In [23]:
print("Status:", model.status)
print("Optimal Solution:")
# Print values of x
for i in I:
    for j in J:
        print(f"x[{i}][{j}] =", x[i][j].varValue)
# Print values of y
for i in I:
    for w in W:
        print(f"y[{i}][{w}] =", y[i][w].varValue)
    
print("Optimal Profit =", model.objective.value())

Status: 1
Optimal Solution:
x[NY][Missi] = 1.0
x[NY][Bramt] = 1.0
x[Rm][Missi] = 0.0
x[Rm][Bramt] = 0.0
x[Mh][Missi] = 0.0
x[Mh][Bramt] = 0.0
x[Ba][Missi] = 0.0
x[Ba][Bramt] = 0.0
x[Va][Missi] = 0.0
x[Va][Bramt] = 0.0
y[NY][NY_reg] = 0.0
y[NY][NY_half] = 1.0
y[Rm][NY_reg] = 0.0
y[Rm][NY_half] = 0.0
y[Mh][NY_reg] = 0.0
y[Mh][NY_half] = 0.0
y[Ba][NY_reg] = 0.0
y[Ba][NY_half] = 0.0
y[Va][NY_reg] = 0.0
y[Va][NY_half] = 0.0
Optimal Profit = 168.5
